In [1]:
import networkx as nx
import numpy as np

import os
from Instance_generator import graph_generator, clustering, tree_gen

Header example

NAME : C :/Sirius/job/Library/LIB/Category - Exact algorithms/Precedence constraints/e1x_1.pcglns
TYPE : PCGLNS
COMMENT : no commen
DIMENSION : 418
GTSP_SETS : 22
EDGE_WEIGHT_TYPE : EXPLICIT
EDGE_WEIGHT_FORMAT : FULL_MATRIX

In [44]:
def make_header(problem_name, n, m):
    header = f'NAME : {problem_name}\nTYPE : PCGLNS\nCOMMENT : no comments\nDIMENSION : {n}\nGTSP_SETS : {m}\nEDGE_WEIGHT_TYPE : EXPLICIT\nEDGE_WEIGHTS_FORMAT : FULL_MATRIX\n'
    return header

def make_edge_weights(G):
    lines=['EDGE_WEIGHT_SECTION']
    n = len(G.nodes)
    
    for v in range(n):
        line = ' '.join([f'{G[v+1][u+1]["weight"]}' if u != v else '0' for u in range(n)])
        lines.append(line)
    return lines

def make_gtsp_sets(clusters):
    lines=['GTSP_SET_SECTION']
    m =len(clusters)
    
    for c_idx in range(1,m+1):
        line = f'{c_idx} ' + ' '.join([f'{v}' for v in clusters[c_idx]]) + ' -1'
        lines.append(line)
        
    return lines

def make_gtsp_sets_ordering(tree, m):
    lines=['GTSP_SET_ORDERING\n']
    for c in range(2,m+1):
        if c in tree:
            desc = nx.descendants(tree,c)
            if desc:
                line = f'{c} ' + ' '.join([f'{d}' for d in desc]) + ' -1\n'
                lines.append(line)
    return lines
            
def make_footer(first_cluster):
    line = f'START_GROUP_SECTION\n{first_cluster}\nEND'
    
    
def main(fname, n, m):
    G = graph_generator(n)
    clusters = clustering(n,m)
    tree = tree_gen(m)
    
    with open(fname, 'r') as fout:
        header = make_header(fname, n, m)
        fout.write(header)
        
        weights_section = make_edge_weights(G)
        fout.write('\n'.join([ln in weights_section]) + '\n')
        
        gtsp_section = make_gtsp_sets(clusters)
        fout.write('\n'.join([ln in gtsp_section]) + '\n')
        
        order_section = make_gtsp_sets_ordering(tree, m)
        fout.write('\n'.join([ln in order_section]) + '\n')
        
        footer = make_footer(1)
        fout.write(footer)

In [3]:
print(make_header('First proba', 100,10))

NAME : First proba
TYPE : PCGLNS
COMMENT : no comments
DIMENSION : 100
GTSP_SETS : 10
EDGE_WEIGHT_TYPE : EXPLICIT
EDGE_WEIGHTS_FORMAT : FULL_MATRIX


In [31]:
mat = np.array(range(25)).reshape(5,5)
G = nx.DiGraph()
n = 5
edges = [(i+1, j+1, mat[i, j]) for i in range(n) for j in range(n) if i != j]
G.add_weighted_edges_from(edges)

lines = make_edge_weights(G)
print(lines)

True
['EDGE_WEIGHT_SECTION\n', '0 1 2 3 4\n', '5 0 7 8 9\n', '10 11 0 13 14\n', '15 16 17 0 19\n', '20 21 22 23 0\n']


In [42]:
clusters = {1: [1,2,3], 2: [4,5], 3: [6,7,8,9], 4: [10,11], 5: [12]}
m = 5
make_gtsp_sets(clusters)

['GTSP_SET_SECTION',
 '1 1 2 3 -1',
 '2 4 5 -1',
 '3 6 7 8 9 -1',
 '4 10 11 -1',
 '5 12 -1']

In [39]:
tree = nx.DiGraph()
m = 5
tree.add_edges_from([[1,2],[1,3],[2,3],[2,4],[3,5]])
lines = make_gtsp_sets_ordering(tree, m)
print(lines)

['GTSP_SET_ORDERING\n', '2 3 4 5 -1\n', '3 5 -1\n']
